In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# ✅ CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ CNN 모델 정의
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 30, kernel_size=5)
        # 12x12 크기로 줄어듦
        # OH = ( ( H - FH + ( 2 * P )  ) // S ) + 1
        # OW 동일
        # ( 28 - 5 + 0 / 1) + 1 = 24
        # 28 x 28이미지가 24 x 24 행렬로 전환됨.
        # 이후 2x2 maxPool계층을 통과하면
        self.pool = nn.MaxPool2d(2, 2)
        # 12 x 12 행렬이 됨.
        self.fc1 = nn.Linear(30 * 12 * 12, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 30 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# ✅ 모델 생성 & CUDA로 이동
model = SimpleConvNet().to(device)

# ✅ 데이터셋 로드 (PyTorch 제공 MNIST)
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=True, download=True,
                   transform=transforms.ToTensor()), batch_size=100, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=False, transform=transforms.ToTensor()), batch_size=1000, shuffle=False
)

# ✅ 옵티마이저 & 손실 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# ✅ 정확도 계산 함수
def compute_accuracy(model, data_loader):
    model.eval()  # 평가 모드
    correct = 0
    total = 0
    with torch.no_grad():  # 역전파 비활성화 (메모리 절약)
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)  # 가장 높은 값의 인덱스 선택
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

# ✅ 학습 루프
epochs = 50  # 학습 횟수 증가
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # GPU로 이동
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # ✅ 학습 & 테스트 정확도 출력
    train_acc = compute_accuracy(model, train_loader)
    test_acc = compute_accuracy(model, test_loader)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print("Training complete!")


Using device: cuda
Epoch 1, Loss: 0.3485, Train Acc: 0.9602, Test Acc: 0.9615
Epoch 2, Loss: 0.1114, Train Acc: 0.9721, Test Acc: 0.9711
Epoch 3, Loss: 0.0707, Train Acc: 0.9831, Test Acc: 0.9808
Epoch 4, Loss: 0.0537, Train Acc: 0.9877, Test Acc: 0.9850
Epoch 5, Loss: 0.0436, Train Acc: 0.9909, Test Acc: 0.9871
Epoch 6, Loss: 0.0366, Train Acc: 0.9898, Test Acc: 0.9860
Epoch 7, Loss: 0.0302, Train Acc: 0.9918, Test Acc: 0.9870
Epoch 8, Loss: 0.0263, Train Acc: 0.9942, Test Acc: 0.9878
Epoch 9, Loss: 0.0233, Train Acc: 0.9929, Test Acc: 0.9861
Epoch 10, Loss: 0.0203, Train Acc: 0.9962, Test Acc: 0.9889
Epoch 11, Loss: 0.0180, Train Acc: 0.9966, Test Acc: 0.9890
Epoch 12, Loss: 0.0148, Train Acc: 0.9974, Test Acc: 0.9882
Epoch 13, Loss: 0.0135, Train Acc: 0.9977, Test Acc: 0.9881
Epoch 14, Loss: 0.0119, Train Acc: 0.9985, Test Acc: 0.9899
Epoch 15, Loss: 0.0103, Train Acc: 0.9982, Test Acc: 0.9896
Epoch 16, Loss: 0.0089, Train Acc: 0.9980, Test Acc: 0.9887
Epoch 17, Loss: 0.0075, Train 

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# ✅ CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ CNN 모델 정의
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 30, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(30 * 12 * 12, 100)  # 12x12 크기로 줄어듦
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 30 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# ✅ 모델 생성 & CUDA로 이동
model = SimpleConvNet().to(device)

# ✅ 데이터셋 로드 (PyTorch 제공 MNIST)
transform = transforms.Compose([
    transforms.RandomRotation(20),  # 20도 회전
    #transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # 기울기 변형 및 크기 조정
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST("./data", train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST("./data", train=False, transform=transforms.ToTensor()), batch_size=1000, shuffle=False)

# ✅ 옵티마이저 & 손실 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 사용

# ✅ 정확도 계산 함수
def compute_accuracy(model, data_loader):
    model.eval()  # 평가 모드
    correct = 0
    total = 0
    with torch.no_grad():  # 역전파 비활성화 (메모리 절약)
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)  # 가장 높은 값의 인덱스 선택
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

# ✅ 학습 루프
epochs = 50  # 학습 횟수 증가
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # GPU로 이동
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # ✅ 학습 & 테스트 정확도 출력
    train_acc = compute_accuracy(model, train_loader)
    test_acc = compute_accuracy(model, test_loader)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print("Training complete!")


Using device: cuda
Epoch 1, Loss: 0.2793, Train Acc: 0.9670, Test Acc: 0.8947
Epoch 2, Loss: 0.1031, Train Acc: 0.9747, Test Acc: 0.8223
Epoch 3, Loss: 0.0804, Train Acc: 0.9772, Test Acc: 0.8970
Epoch 4, Loss: 0.0684, Train Acc: 0.9835, Test Acc: 0.8485
Epoch 5, Loss: 0.0599, Train Acc: 0.9836, Test Acc: 0.8437
Epoch 6, Loss: 0.0563, Train Acc: 0.9856, Test Acc: 0.9188
Epoch 7, Loss: 0.0511, Train Acc: 0.9853, Test Acc: 0.8829
Epoch 8, Loss: 0.0462, Train Acc: 0.9855, Test Acc: 0.9443
Epoch 9, Loss: 0.0438, Train Acc: 0.9870, Test Acc: 0.9021
Epoch 10, Loss: 0.0401, Train Acc: 0.9876, Test Acc: 0.9415
Epoch 11, Loss: 0.0384, Train Acc: 0.9899, Test Acc: 0.9530
Epoch 12, Loss: 0.0366, Train Acc: 0.9903, Test Acc: 0.9445
Epoch 13, Loss: 0.0350, Train Acc: 0.9888, Test Acc: 0.9211
Epoch 14, Loss: 0.0321, Train Acc: 0.9887, Test Acc: 0.9226
Epoch 15, Loss: 0.0292, Train Acc: 0.9921, Test Acc: 0.9551
Epoch 16, Loss: 0.0302, Train Acc: 0.9926, Test Acc: 0.9572
Epoch 17, Loss: 0.0279, Train 

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# ✅ CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ CNN 모델 정의
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 30, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(30 * 12 * 12, 100)  # 12x12 크기로 줄어듦
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 30 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# ✅ 모델 생성 & CUDA로 이동
model = SimpleConvNet().to(device)

# ✅ 데이터셋 로드 (PyTorch 제공 MNIST)
transform = transforms.Compose([
    #transforms.RandomRotation(20),  # 20도 회전
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # 기울기 변형 및 크기 조정
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST("./data", train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST("./data", train=False, transform=transforms.ToTensor()), batch_size=1000, shuffle=False)

# ✅ 옵티마이저 & 손실 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 사용

# ✅ 정확도 계산 함수
def compute_accuracy(model, data_loader):
    model.eval()  # 평가 모드
    correct = 0
    total = 0
    with torch.no_grad():  # 역전파 비활성화 (메모리 절약)
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)  # 가장 높은 값의 인덱스 선택
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

# ✅ 학습 루프
epochs = 50  # 학습 횟수 증가
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # GPU로 이동
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # ✅ 학습 & 테스트 정확도 출력
    train_acc = compute_accuracy(model, train_loader)
    test_acc = compute_accuracy(model, test_loader)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print("Training complete!")


Using device: cuda
Epoch 1, Loss: 0.2413, Train Acc: 0.9654, Test Acc: 0.7923
Epoch 2, Loss: 0.0897, Train Acc: 0.9750, Test Acc: 0.9047
Epoch 3, Loss: 0.0684, Train Acc: 0.9837, Test Acc: 0.9284
Epoch 4, Loss: 0.0564, Train Acc: 0.9845, Test Acc: 0.9338
Epoch 5, Loss: 0.0525, Train Acc: 0.9850, Test Acc: 0.9525
Epoch 6, Loss: 0.0460, Train Acc: 0.9886, Test Acc: 0.9615
Epoch 7, Loss: 0.0409, Train Acc: 0.9877, Test Acc: 0.9478
Epoch 8, Loss: 0.0375, Train Acc: 0.9891, Test Acc: 0.9426
Epoch 9, Loss: 0.0354, Train Acc: 0.9905, Test Acc: 0.9743
Epoch 10, Loss: 0.0327, Train Acc: 0.9904, Test Acc: 0.9562
Epoch 11, Loss: 0.0314, Train Acc: 0.9916, Test Acc: 0.9661
Epoch 12, Loss: 0.0280, Train Acc: 0.9908, Test Acc: 0.9336
Epoch 13, Loss: 0.0258, Train Acc: 0.9937, Test Acc: 0.9568
Epoch 14, Loss: 0.0255, Train Acc: 0.9930, Test Acc: 0.9542
Epoch 15, Loss: 0.0235, Train Acc: 0.9927, Test Acc: 0.9544
Epoch 16, Loss: 0.0235, Train Acc: 0.9942, Test Acc: 0.9562
Epoch 17, Loss: 0.0221, Train 

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# ✅ CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ CNN 모델 정의
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 30, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(30 * 12 * 12, 100)  # 12x12 크기로 줄어듦
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 30 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# ✅ 모델 생성 & CUDA로 이동
model = SimpleConvNet().to(device)

# ✅ 데이터셋 로드 (PyTorch 제공 MNIST)
transform = transforms.Compose([
    transforms.RandomRotation(20),  # 20도 회전
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # 기울기 변형 및 크기 조정
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST("./data", train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST("./data", train=False, transform=transforms.ToTensor()), batch_size=1000, shuffle=False)
# ✅ 옵티마이저 & 손실 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 사용

# ✅ 정확도 계산 함수
def compute_accuracy(model, data_loader):
    model.eval()  # 평가 모드
    correct = 0
    total = 0
    with torch.no_grad():  # 역전파 비활성화 (메모리 절약)
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)  # 가장 높은 값의 인덱스 선택
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

# ✅ 학습 루프
epochs = 50  # 학습 횟수 증가
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # GPU로 이동
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # ✅ 학습 & 테스트 정확도 출력
    train_acc = compute_accuracy(model, train_loader)
    test_acc = compute_accuracy(model, test_loader)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print("Training complete!")


Using device: cuda
Epoch 1, Loss: 0.3123, Train Acc: 0.9548, Test Acc: 0.8960
Epoch 2, Loss: 0.1278, Train Acc: 0.9653, Test Acc: 0.9507
Epoch 3, Loss: 0.1056, Train Acc: 0.9732, Test Acc: 0.9536
Epoch 4, Loss: 0.0906, Train Acc: 0.9739, Test Acc: 0.9000
Epoch 5, Loss: 0.0850, Train Acc: 0.9737, Test Acc: 0.9678
Epoch 6, Loss: 0.0775, Train Acc: 0.9774, Test Acc: 0.9363
Epoch 7, Loss: 0.0736, Train Acc: 0.9797, Test Acc: 0.9651
Epoch 8, Loss: 0.0657, Train Acc: 0.9796, Test Acc: 0.9714
Epoch 9, Loss: 0.0632, Train Acc: 0.9794, Test Acc: 0.9712
Epoch 10, Loss: 0.0606, Train Acc: 0.9817, Test Acc: 0.9758
Epoch 11, Loss: 0.0572, Train Acc: 0.9835, Test Acc: 0.9738
Epoch 12, Loss: 0.0553, Train Acc: 0.9824, Test Acc: 0.9684
Epoch 13, Loss: 0.0552, Train Acc: 0.9828, Test Acc: 0.9759
Epoch 14, Loss: 0.0520, Train Acc: 0.9842, Test Acc: 0.9754
Epoch 15, Loss: 0.0512, Train Acc: 0.9863, Test Acc: 0.9794
Epoch 16, Loss: 0.0465, Train Acc: 0.9839, Test Acc: 0.9844
Epoch 17, Loss: 0.0447, Train 

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# ✅ CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ CNN 모델 정의
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.pool = nn.MaxPool2d(2, 2)  # 28x28 -> 14x14
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # 14x14 -> 14x14
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)  # 14x14 -> 14x14
        self.pool2 = nn.MaxPool2d(2, 2)  # 14x14 -> 7x7
        self.fc1 = nn.Linear(256 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)  # 과적합 방지
        self.batch_norm = nn.BatchNorm1d(512)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = x.view(-1, 256 * 7 * 7)
        x = F.relu(self.batch_norm(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# ✅ 모델 생성 & CUDA로 이동
model = SimpleConvNet().to(device)

# ✅ 데이터셋 로드 (PyTorch 제공 MNIST)
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=True, download=True,
                   transform=transforms.ToTensor()), batch_size=100, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=False, transform=transforms.ToTensor()), batch_size=1000, shuffle=False
)

# ✅ 옵티마이저 & 손실 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 사용

# ✅ 정확도 계산 함수
def compute_accuracy(model, data_loader):
    model.eval()  # 평가 모드
    correct = 0
    total = 0
    with torch.no_grad():  # 역전파 비활성화 (메모리 절약)
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)  # 가장 높은 값의 인덱스 선택
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

# ✅ 학습 루프
epochs = 50  # 학습 횟수 증가
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # GPU로 이동
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # ✅ 학습 & 테스트 정확도 출력
    train_acc = compute_accuracy(model, train_loader)
    test_acc = compute_accuracy(model, test_loader)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print("Training complete!")


Using device: cuda
Epoch 1, Loss: 0.0919, Train Acc: 0.9917, Test Acc: 0.9905
Epoch 2, Loss: 0.0376, Train Acc: 0.9952, Test Acc: 0.9934
Epoch 3, Loss: 0.0260, Train Acc: 0.9964, Test Acc: 0.9945
Epoch 4, Loss: 0.0218, Train Acc: 0.9969, Test Acc: 0.9939
Epoch 5, Loss: 0.0172, Train Acc: 0.9966, Test Acc: 0.9938
Epoch 6, Loss: 0.0144, Train Acc: 0.9974, Test Acc: 0.9934
Epoch 7, Loss: 0.0132, Train Acc: 0.9981, Test Acc: 0.9936
Epoch 8, Loss: 0.0102, Train Acc: 0.9986, Test Acc: 0.9946
Epoch 9, Loss: 0.0102, Train Acc: 0.9983, Test Acc: 0.9934
Epoch 10, Loss: 0.0086, Train Acc: 0.9990, Test Acc: 0.9946
Epoch 11, Loss: 0.0075, Train Acc: 0.9991, Test Acc: 0.9948
Epoch 12, Loss: 0.0071, Train Acc: 0.9994, Test Acc: 0.9951
Epoch 13, Loss: 0.0053, Train Acc: 0.9994, Test Acc: 0.9954
Epoch 14, Loss: 0.0064, Train Acc: 0.9995, Test Acc: 0.9950
Epoch 15, Loss: 0.0059, Train Acc: 0.9991, Test Acc: 0.9948
Epoch 16, Loss: 0.0064, Train Acc: 0.9996, Test Acc: 0.9954
Epoch 17, Loss: 0.0043, Train 

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# ✅ CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ CNN 모델 정의
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.pool = nn.MaxPool2d(2, 2)  # 28x28 -> 14x14
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # 14x14 -> 14x14
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)  # 14x14 -> 14x14
        self.pool2 = nn.MaxPool2d(2, 2)  # 14x14 -> 7x7
        self.fc1 = nn.Linear(256 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)  # 과적합 방지
        self.batch_norm = nn.BatchNorm1d(512)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = x.view(-1, 256 * 7 * 7)
        x = F.relu(self.batch_norm(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# ✅ 모델 생성 & CUDA로 이동
model = SimpleConvNet().to(device)

# ✅ 데이터셋 로드 (PyTorch 제공 MNIST)
transform = transforms.Compose([
    transforms.RandomRotation(20),  # 20도 회전
    #transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # 기울기 변형 및 크기 조정
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST("./data", train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST("./data", train=False, transform=transforms.ToTensor()), batch_size=1000, shuffle=False)
# ✅ 옵티마이저 & 손실 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 사용

# ✅ 정확도 계산 함수
def compute_accuracy(model, data_loader):
    model.eval()  # 평가 모드
    correct = 0
    total = 0
    with torch.no_grad():  # 역전파 비활성화 (메모리 절약)
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)  # 가장 높은 값의 인덱스 선택
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

# ✅ 학습 루프
epochs = 50  # 학습 횟수 증가
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # GPU로 이동
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # ✅ 학습 & 테스트 정확도 출력
    train_acc = compute_accuracy(model, train_loader)
    test_acc = compute_accuracy(model, test_loader)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print("Training complete!")


Using device: cuda
Epoch 1, Loss: 0.1217, Train Acc: 0.9853, Test Acc: 0.2064
Epoch 2, Loss: 0.0567, Train Acc: 0.9828, Test Acc: 0.3905
Epoch 3, Loss: 0.0443, Train Acc: 0.9913, Test Acc: 0.5525
Epoch 4, Loss: 0.0397, Train Acc: 0.9935, Test Acc: 0.5710
Epoch 5, Loss: 0.0343, Train Acc: 0.9922, Test Acc: 0.6720
Epoch 6, Loss: 0.0306, Train Acc: 0.9922, Test Acc: 0.9481
Epoch 7, Loss: 0.0277, Train Acc: 0.9941, Test Acc: 0.8452
Epoch 8, Loss: 0.0258, Train Acc: 0.9940, Test Acc: 0.9546
Epoch 9, Loss: 0.0262, Train Acc: 0.9953, Test Acc: 0.9155
Epoch 10, Loss: 0.0234, Train Acc: 0.9967, Test Acc: 0.9242
Epoch 11, Loss: 0.0205, Train Acc: 0.9966, Test Acc: 0.9037
Epoch 12, Loss: 0.0192, Train Acc: 0.9964, Test Acc: 0.9668
Epoch 13, Loss: 0.0200, Train Acc: 0.9962, Test Acc: 0.9006
Epoch 14, Loss: 0.0180, Train Acc: 0.9973, Test Acc: 0.9513
Epoch 15, Loss: 0.0157, Train Acc: 0.9968, Test Acc: 0.9603
Epoch 16, Loss: 0.0156, Train Acc: 0.9968, Test Acc: 0.9586
Epoch 17, Loss: 0.0137, Train 

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# ✅ CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ CNN 모델 정의
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.pool = nn.MaxPool2d(2, 2)  # 28x28 -> 14x14
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # 14x14 -> 14x14
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)  # 14x14 -> 14x14
        self.pool2 = nn.MaxPool2d(2, 2)  # 14x14 -> 7x7
        self.fc1 = nn.Linear(256 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)  # 과적합 방지
        self.batch_norm = nn.BatchNorm1d(512)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = x.view(-1, 256 * 7 * 7)
        x = F.relu(self.batch_norm(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# ✅ 모델 생성 & CUDA로 이동
model = SimpleConvNet().to(device)

# ✅ 데이터셋 로드 (PyTorch 제공 MNIST)
transform = transforms.Compose([
    #transforms.RandomRotation(20),  # 20도 회전
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # 기울기 변형 및 크기 조정
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST("./data", train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST("./data", train=False, transform=transforms.ToTensor()), batch_size=1000, shuffle=False)
# ✅ 옵티마이저 & 손실 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 사용

# ✅ 정확도 계산 함수
def compute_accuracy(model, data_loader):
    model.eval()  # 평가 모드
    correct = 0
    total = 0
    with torch.no_grad():  # 역전파 비활성화 (메모리 절약)
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)  # 가장 높은 값의 인덱스 선택
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

# ✅ 학습 루프
epochs = 50  # 학습 횟수 증가
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # GPU로 이동
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # ✅ 학습 & 테스트 정확도 출력
    train_acc = compute_accuracy(model, train_loader)
    test_acc = compute_accuracy(model, test_loader)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print("Training complete!")


Using device: cuda
Epoch 1, Loss: 0.1095, Train Acc: 0.9880, Test Acc: 0.5729
Epoch 2, Loss: 0.0484, Train Acc: 0.9895, Test Acc: 0.5109
Epoch 3, Loss: 0.0400, Train Acc: 0.9916, Test Acc: 0.6303
Epoch 4, Loss: 0.0331, Train Acc: 0.9930, Test Acc: 0.9120
Epoch 5, Loss: 0.0307, Train Acc: 0.9940, Test Acc: 0.9215
Epoch 6, Loss: 0.0271, Train Acc: 0.9955, Test Acc: 0.9034
Epoch 7, Loss: 0.0252, Train Acc: 0.9947, Test Acc: 0.8435
Epoch 8, Loss: 0.0236, Train Acc: 0.9956, Test Acc: 0.9221
Epoch 9, Loss: 0.0207, Train Acc: 0.9966, Test Acc: 0.8901
Epoch 10, Loss: 0.0196, Train Acc: 0.9951, Test Acc: 0.9529
Epoch 11, Loss: 0.0195, Train Acc: 0.9955, Test Acc: 0.9588
Epoch 12, Loss: 0.0172, Train Acc: 0.9970, Test Acc: 0.9154
Epoch 13, Loss: 0.0185, Train Acc: 0.9973, Test Acc: 0.9503
Epoch 14, Loss: 0.0158, Train Acc: 0.9966, Test Acc: 0.9581
Epoch 15, Loss: 0.0152, Train Acc: 0.9972, Test Acc: 0.9103
Epoch 16, Loss: 0.0138, Train Acc: 0.9979, Test Acc: 0.9059
Epoch 17, Loss: 0.0134, Train 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# ✅ CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ CNN 모델 정의
class SimpleConvNet(nn.Module):
    class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 28x28 -> 28x28
        self.pool = nn.MaxPool2d(2, 2)  # 28x28 -> 14x14
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # 14x14 -> 14x14
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)  # 14x14 -> 14x14
        self.pool2 = nn.MaxPool2d(2, 2)  # 14x14 -> 7x7
        self.fc1 = nn.Linear(256 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)  # 과적합 방지
        self.batch_norm = nn.BatchNorm1d(512)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = x.view(-1, 256 * 7 * 7)
        x = F.relu(self.batch_norm(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# ✅ 모델 생성 & CUDA로 이동
model = SimpleConvNet().to(device)

# ✅ 데이터셋 로드 (PyTorch 제공 MNIST)
transform = transforms.Compose([
    transforms.RandomRotation(20),  # 20도 회전
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),  # 기울기 변형 및 크기 조정
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST("./data", train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST("./data", train=False, transform=transforms.ToTensor()), batch_size=1000, shuffle=False)
# ✅ 옵티마이저 & 손실 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 사용

# ✅ 정확도 계산 함수
def compute_accuracy(model, data_loader):
    model.eval()  # 평가 모드
    correct = 0
    total = 0
    with torch.no_grad():  # 역전파 비활성화 (메모리 절약)
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)  # 가장 높은 값의 인덱스 선택
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

# ✅ 학습 루프
epochs = 50  # 학습 횟수 증가
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # GPU로 이동
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # ✅ 학습 & 테스트 정확도 출력
    train_acc = compute_accuracy(model, train_loader)
    test_acc = compute_accuracy(model, test_loader)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

print("Training complete!")


Using device: cuda
Epoch 1, Loss: 0.1492, Train Acc: 0.9683, Test Acc: 0.2465
Epoch 2, Loss: 0.0717, Train Acc: 0.9830, Test Acc: 0.3350
Epoch 3, Loss: 0.0569, Train Acc: 0.9875, Test Acc: 0.6807
Epoch 4, Loss: 0.0514, Train Acc: 0.9871, Test Acc: 0.8912
Epoch 5, Loss: 0.0479, Train Acc: 0.9893, Test Acc: 0.8799
Epoch 6, Loss: 0.0408, Train Acc: 0.9884, Test Acc: 0.8868
Epoch 7, Loss: 0.0413, Train Acc: 0.9920, Test Acc: 0.9630
Epoch 8, Loss: 0.0364, Train Acc: 0.9918, Test Acc: 0.9553
Epoch 9, Loss: 0.0369, Train Acc: 0.9923, Test Acc: 0.8952
Epoch 10, Loss: 0.0337, Train Acc: 0.9927, Test Acc: 0.9031
Epoch 11, Loss: 0.0321, Train Acc: 0.9923, Test Acc: 0.9478
Epoch 12, Loss: 0.0305, Train Acc: 0.9933, Test Acc: 0.9295
Epoch 13, Loss: 0.0296, Train Acc: 0.9934, Test Acc: 0.9333
Epoch 14, Loss: 0.0292, Train Acc: 0.9931, Test Acc: 0.9204
Epoch 15, Loss: 0.0285, Train Acc: 0.9937, Test Acc: 0.9364
Epoch 16, Loss: 0.0301, Train Acc: 0.9950, Test Acc: 0.9468
Epoch 17, Loss: 0.0232, Train 